# Packages & Functions

## Packages

In [39]:
import os
import time
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.optim.adam import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Functions

In [40]:
def scailing(x, y):

    ms = MinMaxScaler()
    ss = StandardScaler()

    x_ss = ss.fit_transform(x)
    y_ms = ms.fit_transform(y)

    return x_ss, y_ms

In [121]:
def window_sliding(x, y, iw, ow):
    
    x_ws, y_ws = list(), list()
    for i in range(len(df)):
        x_end = i + iw
        y_end = x_end + ow
        
        if y_end > len(df):
            break
        
        tx = x[i:x_end, :]
        ty = y[x_end:y_end, :]
        
        x_ws.append(tx)
        y_ws.append(ty)
    
    return torch.FloatTensor(x_ws).to(device), torch.FloatTensor(y_ws).to(device)

In [42]:
def plotting(train_loader, test_loader, actual):
    with torch.no_grad():
        train_pred = []
        test_pred = []

        for data in train_loader:
            seq, target = data
            out = model(seq)
            train_pred += out.cpu().numpy().tolist()

        for data in test_loader:
            seq, target = data
            out = model(seq)
            test_pred += out.cpu().numpy().tolist()
      
    total = train_pred + test_pred
    plt.figure(figsize=(20,10))
    plt.plot(np.ones(100)*len(train_pred), np.linspace(0,1,100), '--', linewidth=0.6)
    plt.plot(actual, '--')
    plt.plot(total, 'b', linewidth=0.6)

    plt.legend(['train boundary', 'actual', 'prediction'])
    plt.show()


# Data Processing

## Load Data

In [102]:
data = pd.read_csv('../Data/train.csv')
print(data.dtypes)

Date      object
Open       int64
High       int64
Low        int64
Volume     int64
Close      int64
dtype: object


## Index & Col

In [103]:
df = data.loc[:, ['Date', 'Open']]
df.rename(columns = {'Open':'AC'}, inplace = True)
df['DAC'] = df['AC'] - df['AC'].shift(1)
df['DDAC'] = df['DAC'] - df['DAC'].shift(1)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

In [104]:
df

,AC,DAC,DDAC
Date,,,
2015-12-16,120,NaN,NaN
2015-12-17,124,4.0,NaN
2015-12-18,121,-3.0,-7.0
2015-12-21,120,-1.0,2.0
2015-12-22,117,-3.0,-2.0
...,...,...,...
2019-10-14,284,-1.0,-20.0
2019-10-15,284,0.0,1.0
2019-10-16,283,-1.0,-1.0


## Null Values

In [105]:
df.isnull().sum()

AC      0
DAC     1
DDAC    2
dtype: int64

In [106]:
df.dropna(inplace = True)
df

,AC,DAC,DDAC
Date,,,
2015-12-18,121,-3.0,-7.0
2015-12-21,120,-1.0,2.0
2015-12-22,117,-3.0,-2.0
2015-12-23,117,0.0,3.0
2015-12-24,118,1.0,1.0
...,...,...,...
2019-10-14,284,-1.0,-20.0
2019-10-15,284,0.0,1.0
2019-10-16,283,-1.0,-1.0


## Split Target

In [107]:
x = df.iloc[:, 0:]
y = df.iloc[:,:1]
print(x)
print(y)

             AC   DAC  DDAC
Date                       
2015-12-18  121  -3.0  -7.0
2015-12-21  120  -1.0   2.0
2015-12-22  117  -3.0  -2.0
2015-12-23  117   0.0   3.0
2015-12-24  118   1.0   1.0
...         ...   ...   ...
2019-10-14  284  -1.0 -20.0
2019-10-15  284   0.0   1.0
2019-10-16  283  -1.0  -1.0
2019-10-17  304  21.0  22.0
2019-10-18  289 -15.0 -36.0

[965 rows x 3 columns]
             AC
Date           
2015-12-18  121
2015-12-21  120
2015-12-22  117
2015-12-23  117
2015-12-24  118
...         ...
2019-10-14  284
2019-10-15  284
2019-10-16  283
2019-10-17  304
2019-10-18  289

[965 rows x 1 columns]


## Scailing

In [110]:
ms = MinMaxScaler()
ss = StandardScaler()

x_ss = ss.fit_transform(x)
y_ms = ms.fit_transform(y)

In [111]:
x_ss, y_ms

(array([[-0.98782528, -0.46905606, -0.69755913],
        [-0.99740329, -0.17321351,  0.20183235],
        [-1.02613732, -0.46905606, -0.19789719],
        ...,
        [ 0.56381226, -0.17321351, -0.09796481],
        [ 0.76495046,  3.0810545 ,  2.20048009],
        [ 0.62128032, -2.24411134, -3.59559834]]),
 array([[0.11764706],
        [0.11470588],
        [0.10588235],
        [0.10588235],
        [0.10882353],
        [0.10588235],
        [0.10882353],
        [0.11176471],
        [0.10294118],
        [0.08235294],
        [0.08529412],
        [0.07058824],
        [0.10294118],
        [0.10294118],
        [0.09117647],
        [0.10294118],
        [0.09705882],
        [0.07352941],
        [0.06176471],
        [0.07647059],
        [0.08235294],
        [0.07941176],
        [0.07058824],
        [0.05588235],
        [0.05588235],
        [0.04705882],
        [0.03823529],
        [0.03235294],
        [0.03235294],
        [0.04411765],
        [0.03235294],
        [

In [113]:
x = x.to_numpy()
y = y.to_numpy()
x, y = window_sliding(x, y, 60, 7)
x_ss, y_ms = window_sliding(x_ss, y_ms, 60, 7)

In [122]:
x_train = x_ss[:650]
y_train = y_ms[:650]
x_test = x_ss[650:]
y_test = y_ms[650:]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

torch.Size([650, 60, 3])
torch.Size([650, 7, 1])
torch.Size([249, 60, 3])
torch.Size([249, 7, 1])


In [138]:
train = torch.utils.data.TensorDataset(x_train, y_train)
test = torch.utils.data.TensorDataset(x_test, y_test)

batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = False)

# Model

## Define Model

In [140]:
class lstm_encoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)

    def forward(self, x_input):
        lstm_out, self.hidden = self.lstm(x_input)
        return lstm_out, self.hidden

In [141]:
class lstm_decoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers = 1):
        super(lstm_decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size = input_size,
                            hidden_size = hidden_size,
                            num_layers = num_layers,
                            batch_first = True)
        self.linear = nn.Linear(hidden_size, input_size)
        
    def forward(self, x_input, encoder_hidden_states):
        lstm_out, self.hidden = self.lstm(x_input.unsqueeze(-1), encoder_hidden_states)
        output = self.linear(lstm_out)
        
        return output, self.hidden

In [142]:
class lstm_encoder_decoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(lstm_encoder_decoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.encoder = lstm_encoder(input_size = input_size,
                                    hidden_size = hidden_size)
        self.decoder = lstm_decoder(input_size = input_size,
                                    hidden_size = hidden_size)
        
    def forward(self, inputs, targets, target_len, teacher_forcing_ratio):
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]
        
        outputs = torch.zeros(batch_size, target_len, input_size)
        
        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:, -1, :]
        
        for t in range(target_len):
            out, hidden = self.decoder(decoder_input, hidden)
            out = out.squeeze(1)
            
            if random.random() < teacher_forcing_ratio:
                decoder_input = targets[:, t, :]
            else:
                decoder_input = out
            outputs[:, t, :] = out
            
        return outputs
    
    def predict(self, inputs, target_len):
        self.eval()
        inputs = inputs.unsqueeze(0)
        batch_size = inputs.shape[0]
        input_size = inputs.shape[2]
        outputs = torch.zeros(batch_size, target_len, input_size)
        _, hidden = self.encoder(inputs)
        decoder_input = inputs[:, -1, :]
        for t in range(target_len):
            out, hidden = self.decoder(decoder_input, hidden)
            out = out.squeeze(1)
            decoder_input = out
            outputs[:, t, :] = out
        
        return outputs.detach().numpy()[0, :, 0]

In [143]:
input_size = x_ss.size(2)
num_layers = 2
hidden_size = 8
sequence_length = 60

In [144]:
model = lstm_encoder_decoder(input_size = input_size,
           hidden_size = hidden_size).to(device)

criterion = nn.MSELoss()
lr = 1e-4
num_epochs = 1000
optimizer = Adam(model.parameters(), lr = lr)
patience = 10

## Training

In [145]:
from tqdm import tqdm

model.train()

for x,y in train_loader:
    optimizer.zero_grad()
    x = x.to(device).float()
    y = y.to(device).float()
    output = model(x, y, 7, 0.6).to(device)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
    total_loss += loss.cpu().item()
tr.set_postfix(loss="{0:.5f}".format(total_loss/len(train_loader)))

RuntimeError: input.size(-1) must be equal to input_size. Expected 3, got 1

In [139]:
seq.shape

torch.Size([32, 60, 3])